# ws 02 03

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = './train_set'
val_dir = './val_set'

test_dir = './test_set'

target_img_shape=(64, 64) 

train_datagen = ImageDataGenerator(rescale = 1./255,  # 
        rotation_range=20,
        height_shift_range=0.15,
        width_shift_range=0.15, 
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# train_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size=target_img_shape,
                                                 batch_size=32, 
                                                 class_mode='binary') 

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size=target_img_shape,
                                            batch_size=32,
                                            class_mode='binary')

In [ ]:
print('Training')
ids, counts = np.unique(train_set.classes, return_counts=True)

print(ids)
print(counts)

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels  

for i in ids:
    print('{:>6} = {}' . format(labels[i], counts[i]))

In [ ]:
print('Validation')
ids, counts = np.unique(val_set.classes, return_counts=True)

print(ids)
print(counts)

In [ ]:
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels   #  labels[2] = 'elephant'

for i in ids:
    print('{:>6} = {}' . format(labels[i], counts[i]))

In [ ]:

_, train_count = np.unique(train_set.classes, return_counts=True)
_, val_count = np.unique(val_set.classes, return_counts=True)

print('Ratio Validation/Training set:', 
      val_count/(train_count+val_count) * 100)

In [ ]:
train_set[0][0][0].shape

In [ ]:
print(train_set.class_indices)

for image_batch, labels_batch in train_set:
    print(image_batch.shape)
    print(labels_batch.shape)
    plt.imshow(image_batch[0])
    print('class:', labels_batch[0])
    
    break

In [ ]:
plt.imshow(train_set[0][0][0])
plt.show()

In [ ]:
plt.imshow(val_set[0][0][0])
plt.show()

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 4, figsize=(10,10))
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_set[0][0][0] for i in range(4)]
plotImages(augmented_images)

In [ ]:
nplots = 8

def visual_multi(images_arr):
    fig = plt.figure(figsize=(11, 8)) 
    for j in range(nplots):

        plt.subplot(3, 4, j+1)
        plt.imshow(images_arr[j])

        plt.axis('off')

    plt.show()


augmented_images = [train_set[0][0][0] for i in range(nplots)]
visual_multi(augmented_images)

# Model

In [ ]:
in_shape=(target_img_shape[0],target_img_shape[1],3)
in_shape

In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, Conv2D, AveragePooling2D, 
Flatten, Dropout, MaxPool2D )

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=in_shape)) 
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(128, (3,3), activation='relu')) 
model.add(MaxPool2D((2, 2)))
model.add(Flatten())

model.add(Dense(128, activation='relu'))  # 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import time
start = time.time()

history = model.fit(train_set, steps_per_epoch=len(train_set),
                     validation_data=val_set, 
                    epochs=40, verbose=1) 

end = time.time()
print("Time Taken: {:.2f} minutes".format((end - start)/60))

In [ ]:
plt.figure(figsize=(10, 3.5))
plt.subplot(1, 2, 1)

plt.title('Loss')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'],'r', lw=3.2, label='Validation loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.title('Accuracy')

plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], 'r', lw=3.2, label='Validation')
plt.legend()
plt.show()

In [ ]:
acc = model.evaluate(val_set, steps=len(val_set), verbose=0)
print('score = {:.3f}' .format(acc[1]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

target_img_shape=(64, 64)

test_image = image.load_img('../dog-2.jpg', target_size=target_img_shape)  # PIL

test_image = image.img_to_array(test_image)
test_image /= 255.0

plt.tight_layout()
plt.imshow(test_image)
plt.show()

In [ ]:
test_image = np.expand_dims(test_image, axis=0)
test_image.shape

In [ ]:
result = model.predict(test_image)
result

In [ ]:
train_set.class_indices

In [ ]:
if result[0][0] > 0.5:
    predict='Dog'
else:
    predict='Cat'

print(predict)

In [ ]:
cls = 'Dog' if result[0][0] > 0.5 else 'Cat'
cls

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_dog_cat(lst):
    y_pred = [] ; y_pred_cls = []; img_lst = []
    for i in lst:
        
        img = load_img(i, target_size=target_img_shape)
        img = img_to_array(img)
        img /= 255.0
      
        
        img_lst.append(img)

        img = np.expand_dims(img, axis=0)
    
        y_pred_i = model.predict(img)
        y_pred_cls_i = 'Dog' if y_pred_i > 0.5 else 'Cat'  # 
#         if y_pred_i[0] > 0.5:
#             y_pred_cls_i = 'Dog'
#         else:
#             y_pred_cls_i = 'Cat'
            
        y_pred.append(y_pred_i)
        y_pred_cls.append(y_pred_cls_i)

    return img_lst, y_pred_cls, y_pred

In [ ]:
import glob

mylist = [f for f in glob.glob('./test_predict/*')]

img_lst,y_pred_cls,y_pred = predict_dog_cat(mylist) 

mylist
for i in mylist:
    print(os.path.basename(i), end=' | ')

In [ ]:
nplots = 10
fig = plt.figure(figsize=(10, 5)) 

for i, k in enumerate(img_lst):

    plt.subplot(nplots//5, 5, i+1)
    plt.imshow(k, cmap=plt.cm.gray_r)
    plt.title('p--> {} {}'.format(y_pred_cls[i],y_pred[i][0].round(3)))
    
    fname = os.path.basename(mylist[i])
    plt.title('p--> {} {}\n{}'.format(y_pred_cls[i],y_pred[i][0].round(3), fname))

    plt.xticks([])
    plt.yticks([])

    if i >= nplots-1:  # 
        break

plt.show()